# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """ 

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed = Website("https://miladsoltany.com")
print(ed.links)
print(ed.get_contents())

['/', '/', '/milads-cv', '/', '/', '/milads-cv', 'https://www.linkedin.com/', 'mailto:email@email.com', 'https://scholar.google.ca/citations?user=Gm23tVgAAAAJ&hl=en', 'https://github.com/miladsoltany', 'https://assets.zyrosite.com/m2WQJMxWN7SQpz4E/milad_cv_2024-2-YX4bQn5pjVT5PBKo.pdf']
Webpage Title:
Milad Soltany: Data Scientist and Machine Learning Researcher | Milad Soltany
Webpage Contents:
Home
Milad's CV
Home
Milad's CV
Milad Soltany
Data Scientist and Machine Learning Engineer
Milad's CV
About Milad
Data Scientist and Machine Learning Engineer
Experienced Data Scientist and Machine Learning researcher and educator specializing in Computer Vision, Autonomous Vehicles, and Data Science with a strong academic background and extensive teaching experience.
I love to teach!
Since 2018, I’ve had the privilege of teaching over 2,000 students how to code in Python, guiding them through the fundamentals and advanced concepts of programming. Along with coding, I’ve introduced them to the w

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [14]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://miladsoltany.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/
/milads-cv
/
/
/milads-cv
https://www.linkedin.com/
mailto:email@email.com
https://scholar.google.ca/citations?user=Gm23tVgAAAAJ&hl=en
https://github.com/miladsoltany
https://assets.zyrosite.com/m2WQJMxWN7SQpz4E/milad_cv_2024-2-YX4bQn5pjVT5PBKo.pdf


In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
get_links('https://miladsoltany.com')

{'links': [{'type': 'home page', 'url': 'https://miladsoltany.com'},
  {'type': 'resume or CV', 'url': 'https://miladsoltany.com/milads-cv'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/'},
  {'type': 'GitHub profile', 'url': 'https://github.com/miladsoltany'},
  {'type': 'CV PDF',
   'url': 'https://assets.zyrosite.com/m2WQJMxWN7SQpz4E/milad_cv_2024-2-YX4bQn5pjVT5PBKo.pdf'}]}

In [15]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/Datou1111/shou_xin',
 '/tencent/HunyuanVideo',
 '/CohereForAI/c4ai-command-r7b-12-2024',
 '/black-forest-labs/FLUX.1-dev',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/multimodalart/flux-style-shaping',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/aiqtech/kolaw',
 '/datasets/argilla/FinePersonas-v0.1',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers

In [16]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [5]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on

In [11]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [6]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\n6 days ago\n•\n166k\n•\n1.09k\nDatou1111/shou_xin\nUpdated\n8 days ago\n•\n18.1k\n•\n477\ntencent/HunyuanVideo\nUpdated\n10 days ago\n•\n4.61k\n•\n1.07k\nCohereForAI/c4ai-command-r7b-12-2024\nUpdated\nabout 14 hours ago\n•\n1.84k\n•\n218\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.36M\n•\n7.31k\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.48k\n🏢\nTRELLIS\nScalable and Versat

In [7]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [15]:
create_brochure("BBC.com", "https://bbc.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.bbc.co.uk/aboutthebbc'}, {'type': 'business page', 'url': 'https://www.bbc.com/business'}, {'type': 'innovation page', 'url': 'https://www.bbc.com/innovation'}, {'type': 'culture page', 'url': 'https://www.bbc.com/culture'}, {'type': 'arts page', 'url': 'https://www.bbc.com/arts'}, {'type': 'travel page', 'url': 'https://www.bbc.com/travel'}, {'type': 'future planet page', 'url': 'https://www.bbc.com/future-planet'}, {'type': 'careers page', 'url': 'https://www.bbc.co.uk/contact'}, {'type': 'main website', 'url': 'https://www.bbc.com'}]}


# BBC Company Brochure

---

## Welcome to BBC

### About Us
The British Broadcasting Corporation (BBC) stands at the forefront of global news, culture, and entertainment. With a longstanding commitment to providing impartial information, BBC is dedicated to informing, educating, and entertaining audiences around the world. Our extensive coverage ranges from breaking news and world events to in-depth journalism on culture, business, science, and innovations shaping our future.

### Our Services
- **News Coverage:** Stay informed with our comprehensive reporting on current affairs, US & Canada news, UK politics, and international coverage spanning continents including Africa, Asia, and Europe.
- **Sports:** Get the latest updates on various sporting events and analyses from BBC Sport.
- **Business & Innovation:** Explore the intersection of innovation and business, featuring insights into technology, environmental issues, and future-forward solutions.
- **Culture & Arts:** Dive into a creative world through our dedicated segments on film, TV, music, art, and design.
- **Travel:** Discover travel adventures across the globe with expert recommendations and features.
- **Science & Environment:** Engage with our coverage on climate solutions, sustainable living, and groundbreaking scientific discoveries.

### Company Culture
At BBC, we foster a culture of inclusivity, collaboration, and creativity. Our team consists of passionate professionals dedicated to delivering high-quality content. We encourage open communication and innovation while emphasizing integrity and the public service ethos. 

### Our Audience
With millions of viewers globally, our audience includes diverse individuals seeking reliable news, rich cultural experiences, and educational content. We prioritize producing content that caters to a variety of perspectives, making information accessible to all.

### Careers at BBC
The BBC is known for providing exciting career opportunities across a wide range of fields, including journalism, technology, production, and research. We value talent and offer a supportive environment for growth and development. Join our team and contribute to shaping the future of media and storytelling.

#### Current Vacancies
- Journalists (Various Political and Cultural Focus Areas)
- Digital Content Creators
- Data Analysts
- Video Production Specialists
- Marketing and Outreach Coordinators

### Join Us
Whether you’re a customer looking for trustworthy news, an investor seeking to partner with a reputable organization, or a recruit hoping to become part of our esteemed team, the BBC welcomes you. Together, let’s continue to share stories that shape our world.

---

*For more information, visit [BBC.com](https://www.bbc.com) or contact our careers team to explore job opportunities.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [16]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## About Us
**Hugging Face** is a leading platform and community dedicated to advancing artificial intelligence and machine learning for everyone. We are committed to building a future where collaboration drives innovation. Our platform serves as a home for models, datasets, and applications, enabling machine learning enthusiasts and professionals to create, share, and learn from one another.

## Our Offerings
- **Models:** With over 400,000 models available, Hugging Face provides cutting-edge implementations from top organizations like Microsoft, Google, and Amazon.
- **Datasets:** Access more than 100,000 datasets to enhance your ML projects.
- **Spaces:** Collaborate in real-time by hosting and running applications seamlessly.
- **Enterprise Solutions:** We cater to over 50,000 organizations by offering enterprise-grade solutions, enhanced security, and dedicated support.

## Community Engagement
Hugging Face thrives on collaboration. We provide tools like **Transformers**, **Diffusers**, and **Tokenizers** to help you build robust machine learning applications efficiently. Our community of developers and researchers actively contribute by sharing insights, models, and code, making breakthroughs accessible to all.

## Company Culture
At Hugging Face, we embody inclusivity and innovation. Our team believes in the open source philosophy, transparency, and the power of collective knowledge. We are dedicated to nurturing a community where diverse perspectives lead to impactful solutions in AI. 

## Careers at Hugging Face
We are constantly seeking passionate individuals who want to contribute to the future of AI. Working at Hugging Face means joining a collaborative and dynamic environment that values creativity and personal growth. Explore our [Careers Page](#) to learn more about exciting opportunities to join our team.

## Join Us
Become part of the AI community that is shaping the future. Whether you're a developer, researcher, or simply someone curious about machine learning, **Sign Up** today to unlock a world of possibilities.

[Explore Our Platform](#)

---

**Connect with Us:**
- [GitHub](#)
- [Twitter](#)
- [LinkedIn](#)
- [Discord](#)

### Hugging Face — The AI community building the future.


In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>